In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your data from the 'Cleaned_data.txt' file into a DataFrame
data = pd.read_csv('./Cleaned_Data.txt', sep=', ',  header=None, names=['Image1', 'Image2', 'Label'])

# Split the data into training (70%), validation (15%), and testing (15%) sets
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Now you have your data split into training, validation, and testing sets
# You can access the data and labels as follows:
train_images1, train_images2, train_labels = train_data['Image1'], train_data['Image2'], train_data['Label']
val_images1, val_images2, val_labels = val_data['Image1'], val_data['Image2'], val_data['Label']
test_images1, test_images2, test_labels = test_data['Image1'], test_data['Image2'], test_data['Label']


/var/folders/0y/926bml0x27j90kg_6jr1s2h00000gn/T/ipykernel_25345/2401875523.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('./Cleaned_Data.txt', sep=', ',  header=None, names=['Image1', 'Image2', 'Label'])


In [1]:
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import cv2
import numpy as np
import pandas as pd



# Load a pre-trained Haar Cascade Classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to detect and align faces in an image and save them
def detect_and_align_faces(image_path, output_folder):
    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Convert the image to grayscale for face detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    aligned_face_paths = []

    # Loop through detected faces
    for i, (x, y, w, h) in enumerate(faces):
        # Extract the detected face region
        face = image[y:y+h, x:x+w]

        # Resize the face to a common size for alignment (e.g., 160x160 pixels)
        aligned_face = cv2.resize(face, (160, 160))

        # Generate a unique filename for the aligned face image
        aligned_face_filename = f"{i}_{os.path.basename(image_path)}"
        aligned_face_path = os.path.join(output_folder, aligned_face_filename)

        # Save the aligned face image
        cv2.imwrite(aligned_face_path, aligned_face)

        aligned_face_paths.append(aligned_face_path)

    return aligned_face_paths

# Read the original data from the text file
data = pd.read_csv('./Cleaned_Data.txt', sep=', ',  header=None, names=['Image1', 'Image2', 'Label'])

# Create a new DataFrame for the processed data
new_data = pd.DataFrame(columns=['Image1', 'Image2', 'Label'])

# Process each row in the original data
for index, row in data.iterrows():
    image1_path = row['Image1']
    image2_path = row['Image2']
    label = row['Label']

    # Detect and align faces for Image1 and Image2, saving them in the "train_processed" folder
    aligned_faces1 = detect_and_align_faces(image1_path, './train_processed')
    aligned_faces2 = detect_and_align_faces(image2_path, './train_processed')

    # Create new rows in the DataFrame for each aligned face pair
    for face1, face2 in zip(aligned_faces1, aligned_faces2):
        new_data = new_data.append({'Image1': face1, 'Image2': face2, 'Label': label}, ignore_index=True)

# Save the processed data to a new text file
new_data.to_csv('./Processed_data.txt', sep=',',  header=None, index=False)


/var/folders/0y/926bml0x27j90kg_6jr1s2h00000gn/T/ipykernel_32013/3691110440.py:44: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('./Cleaned_Data.txt', sep=', ',  header=None, names=['Image1', 'Image2', 'Label'])
/var/folders/0y/926bml0x27j90kg_6jr1s2h00000gn/T/ipykernel_32013/3691110440.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append({'Image1': face1, 'Image2': face2, 'Label': label}, ignore_index=True)
/var/folders/0y/926bml0x27j90kg_6jr1s2h00000gn/T/ipykernel_32013/3691110440.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append({'Image1': fa

KeyboardInterrupt: 

In [2]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow as tf

# Configuration for GPU memory allocation
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)


# Load a pre-trained Haar Cascade Classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the pre-trained FaceNet model
facenet = cv2.dnn.readNetFromTensorflow('./facenet-master')

# Function to detect and align faces in an image and extract embeddings
def detect_align_and_extract_embeddings(image_path, output_folder):
    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Convert the image to grayscale for face detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    embeddings = []

    # Loop through detected faces
    for i, (x, y, w, h) in enumerate(faces):
        # Extract the detected face region
        face = image[y:y+h, x:x+w]

        # Resize the face to a common size for embedding extraction (e.g., 160x160 pixels)
        aligned_face = cv2.resize(face, (160, 160))

        # Preprocess the aligned face for FaceNet
        blob = cv2.dnn.blobFromImage(aligned_face, 1.0 / 127.5, (160, 160), (127.5, 127.5, 127.5), swapRB=True)

        # Pass the preprocessed face through FaceNet to obtain embeddings
        facenet.setInput(blob)
        embedding = facenet.forward()

        # Save the aligned face image
        aligned_face_filename = f"{i}_{os.path.basename(image_path)}"
        aligned_face_path = os.path.join(output_folder, aligned_face_filename)
        cv2.imwrite(aligned_face_path, aligned_face)

        # Append the embedding to the list
        embeddings.append({'Image1': aligned_face_path, 'Embedding': embedding.flatten()})

    return embeddings

# Read the original data from the text file
data = pd.read_csv('Cleaned_data.txt', delimiter='\t', header=None, names=['Image1', 'Image2', 'Label'])

# Create a new DataFrame for the processed data
new_data = pd.DataFrame(columns=['Image1', 'Image2', 'Label', 'Embedding'])

# Process each row in the original data
for index, row in data.iterrows():
    image1_path = row['Image1']
    image2_path = row['Image2']
    label = row['Label']

    # Detect, align, and extract embeddings for Image1 and Image2, saving them in the "train_processed" folder
    embeddings1 = detect_align_and_extract_embeddings(image1_path, 'train_processed')
    embeddings2 = detect_align_and_extract_embeddings(image2_path, 'train_processed')

    # Create new rows in the DataFrame for each aligned face pair and their embeddings
    for emb1, emb2 in zip(embeddings1, embeddings2):
        new_data = new_data.append({'Image1': emb1['Image1'], 'Image2': emb2['Image1'], 'Label': label, 'Embedding': emb1['Embedding']}, ignore_index=True)

# Save the processed data to a new text file
new_data.to_csv('Data_with_embeddings.txt', sep=',', header=False, index=False)


: 